<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

<center><h1>Evaluations Quickstart </h1></center>

## Overview
Evaluations are essential to understanding how well your model is performing in real-world scenarios, allowing you to identify strengths, weaknesses, and areas of improvement.

Offline evaluations are run as code and then sent back to Arize using `log_evaluations_sync`.

This guide assumes you have traces in Arize and are looking to run an evaluation to measure your application performance.

To add evaluations you can set up online evaluations as a task to run automatically, or you can follow the steps below to generate evaluations and log them to Arize:

1. Install the Arize SDK
2. Import your spans in code
3. Run a custom evaluator using Phoenix
4. Log evaluations back to Arize

## Install dependencies and setup keys

In [ ]:
!pip install -q arize arize-phoenix openai pandas nest_asyncio 'httpx<0.28'

Copy the ARIZE_API_KEY, SPACE_ID, and ARIZE_DEVELOPER_KEY from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [ ]:
import os
from getpass import getpass

if not os.environ.get("SPACE_ID"):
    os.environ["SPACE_ID"] = getpass("🔑 Enter your space id: ")

if not os.environ.get("ARIZE_API_KEY"):
    os.environ["ARIZE_API_KEY"] = getpass("🔑 Enter your ARIZE_API_KEY: ")

if not os.environ.get("ARIZE_DEVELOPER_KEY"):
    os.environ["ARIZE_DEVELOPER_KEY"] = getpass("🔑 Enter your developer key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

## Import your spans in code

Once you have traces in Arize, you can visit the LLM Tracing tab to see your traces and export them in code. By clicking the export button, you can get the boilerplate code to copy paste to your evaluator.

In [ ]:
# import statements required for getting your spans
from datetime import datetime, timedelta
from arize.exporter import ArizeExportClient
from arize.utils.types import Environments

start_time = datetime.now() - timedelta(days=14)  # 14 days ago
end_time = datetime.now()  # Today

# Exporting your dataset into a dataframe
client = ArizeExportClient(api_key=os.environ["ARIZE_DEVELOPER_KEY"])
primary_df = client.export_model_to_df(
    space_id=os.environ["SPACE_ID"],
    model_id="tracing-haiku-tutorial",  # change this to the name of your project
    environment=Environments.TRACING,
    start_time=start_time,
    end_time=end_time,
)

## Run a custom evaluator using Phoenix

Create a prompt template for the LLM to judge the quality of your responses. You can utilize any of the Arize Evaluator Templates or you can create your own. Below is an example which judges the positivity or negativity of the LLM output.

In [ ]:
import os
from phoenix.evals import OpenAIModel, llm_classify

eval_model = OpenAIModel(
    model="gpt-4o", temperature=0, api_key=os.environ["OPENAI_API_KEY"]
)

MY_CUSTOM_TEMPLATE = """
    You are evaluating the positivity or negativity of the responses to questions.
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Response]: {output}
    [END DATA]


    Please focus on the tone of the response.
    Your answer must be single word, either "positive" or "negative"
    """

Notice the variables in brackets for {input} and {output} above. You will need to set those variables appropriately for the dataframe so you can run your custom template. We use OpenInference as a set of conventions (complementary to OpenTelemetry) to trace AI applications. This means depending on the provider you are using, the attributes of the trace will be different.

You can use the code below to check which attributes are in the traces in your dataframe.

In [ ]:
primary_df.columns

Use the code below to set the input and output variables needed for the prompt above.

In [ ]:
primary_df["input"] = primary_df["attributes.input.value"]
primary_df["output"] = primary_df["attributes.output.value"]

Use the `llm_classify` function to run the evaluation using your custom template. You will be using the dataframe from the traces you generated above. We also add `nest_asyncio` to run the evaluations concurrently (if you are running multiple evaluations).

In [ ]:
import nest_asyncio

nest_asyncio.apply()

evals_df = llm_classify(
    dataframe=primary_df,
    template=MY_CUSTOM_TEMPLATE,
    model=eval_model,
    rails=["positive", "negative"],
    provide_explanation=True,
)

If you'd like more information, see our detailed guide on <a href="https://docs.arize.com/arize/llm-evaluation-and-annotations/catching-hallucinations/custom-evaluators"> custom evaluators.</a> You can also use our <a href="https://docs.arize.com/arize/llm-evaluation-and-annotations/catching-hallucinations/arize-evaluators-llm-as-a-judge">pre-tested evaluators</a> for evaluating hallucination, toxicity, retrieval, etc.

# Log evaluations back to Arize
Use the `log_evaluations_sync` function as part of our Python SDK to attach evaluations you've run to traces. The code below assumes that you have already completed an evaluation run, and you have the `evals_dataframe` object. It also assumes you have a `traces_dataframe` object to get the `span_id` that you need to attach the evals.

The `evals_dataframe` requires four columns, which should be auto-generated for you based on the evaluation you ran using Phoenix. The `<eval_name>` must be alphanumeric and cannot have hyphens or spaces.
- `eval.<eval_name>.label`
- `eval.<eval_name>.score`
- `eval.<eval_name>.explanation`
- `context.span_id`

An example evaluation data dictionary would look like:
```python
evaluation_data = {
   'context.span_id': ['74bdfb83-a40e-4351-9f41-19349e272ae9'],  # Use your span_id
   'eval.myeval.label': ['accuracy'],  # Example label name
   'eval.myeval.score': [0.95],        # Example label value
   'eval.myeval.explanation': ["some explanation"]
}
evaluation_df = pd.DataFrame(evaluation_data)
```

Here is sample code to log the evaluations back to Arize. The API reference can be found <a href="https://arize-client-python.readthedocs.io/en/latest/llm-api/logger.html#arize.pandas.logger.Client.log_evaluations_sync">here</a>.

In [ ]:
evals_df["eval.tone_eval.label"] = evals_df["label"]
evals_df["eval.tone_eval.explanation"] = evals_df["explanation"]
evals_df.head()

In [ ]:
import os
from arize.pandas.logger import Client

ARIZE_API_KEY = os.environ.get("ARIZE_API_KEY")
SPACE_ID = os.environ.get("SPACE_ID")
ARIZE_DEVELOPER_KEY = os.environ.get("ARIZE_DEVELOPER_KEY")

# Initialize Arize client to log evaluations
arize_client = Client(
    space_id=SPACE_ID, api_key=ARIZE_API_KEY, developer_key=ARIZE_DEVELOPER_KEY
)

# Set the evals_df to have the correct span ID to log it to Arize
evals_df["context.span_id"] = primary_df["context.span_id"]

# send the eval to Arize
arize_client.log_evaluations_sync(evals_df, "tracing-haiku-tutorial")